In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    default_data_collator,
    SchedulerType
)
import os
import json
from itertools import chain
from datasets import load_dataset

In [2]:
block_size = 2048
train_file = 'combine.jsonl'
tokenizer = AutoTokenizer.from_pretrained(
    'meta-llama/Llama-2-7b-hf',
)
text_column_name = 'text'

In [ ]:
raw_datasets = load_dataset(
    'json',
    data_files=train_file,
    split='train'
)

filename = os.path.split(train_file)[1]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

column_names = raw_datasets.column_names
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-tokenized-{block_size}',
    num_proc=20,
)

def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-grouped-{block_size}',
    num_proc=20,
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=20):   0%|          | 0/33339118 [00:00<?, ? examples/s]